#RNN을 이요한 텍스트 생성
문맥을 반영해서 다음 단어를 예측하여 텍스트를 생성하는 것 - 다항분류
텍스트 제너레이션은 분류임

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences, to_categorical
import numpy as np
from keras.layers import Embedding, Dense, LSTM, Flatten
from keras.models import Sequential

'''
text="""
거리에 하얀 눈이 쌓이고 있다
그의 눈이 밝게 빛난다
눈이 내리고 있는 오후 거리는 눈이 부시다
"""
'''
text='''
영국 매체 ‘디 애슬레틱’은 3일(한국 시간) “미켈 아르테타 감독은 아스널이 울버햄튼 원더러스와의 경기에서 2-1로 승리하는 과정에서 교체된 토미야스 타케히로를 평가할 것이라고 말했다”라고 보도했다.
토미야스는 양발을 모두 잘 쓰는 멀티 플레이어 수비수다. 기본기를 강조하는 일본 출신 선수답게 기본적인 발밑 기술이 잘 잡혀있는데, 이를 토미야스는 모두 양발로 자유롭게 구사할 수 있다. 이 때문에 팀의 후방 빌드업이 매끄럽게 진행되는데 많은 기여를 한다. 또한 라이트백과 레프트백, 센터백까지 소화가 가능하다.
2021년 아스널로 이적한 그는 아스널 역사상 5번째 아시아인이자 4번째 일본인 선수이며, 즉시 전력감 아시아 선수로는 토미야스는 박주영에 이은 두 번째 선수였다.
2021/22 시즌에 토미야스는 전반기의 활약상은 훌륭했으나, 후반기는 부상과 코로나 등의 문제로 대부분 날려 먹었다. 특히 아스널이 한창 순위 싸움을 치열하게 벌이던 막바지에 부상으로 결장한 것은 팀에 큰 부담이 되었다. 하지만 나오기만 하면 좋은 모습을 보여줬기에 팬들의 기대치는 높았다.
지난 시즌에도 주전으로는 활약하지 못했지만 백업으로 나오면서 좋은 모습을 보여줬다. 하지만 또 부상이 그의 발목을 잡았고 결장 기간이 점점 더 늘어나면서 팬들도 조금은 지쳐갔다.
이번 시즌에는 교체와 선발을 번갈아 가면서 나오며 아주 좋은 모습을 보여주고 있다. 특히 10월에 5경기에 출전하는 동안 인상적인 활약을 펼쳤으며, 데클란 라이스, 가브리엘 제주스, 부카요 사카를 제치고 64%의 득표율로 이달의 선수로 선정되며 아스널 팬들의 인정을 받았다.
구단은 이 상을 발표하면서, 토미야스가 맨체스터 시티와의 경기에서 교체로 나와서 승리에 핵심적인 역할을 수행했다는 점과 유럽대항전 승리에 기여했다는 점을 꼽았다. 게다가 그가 10월에 12번의 리커버리와 8번의 경합 승리를 기록했다고 덧붙였다.
토미야스는 다시 한번 좋은 모습을 보여주며 주전으로 자리 잡나 싶어지만 또 부상당했다. 토미야스는 지난 3일(한국 시간) 울버햄튼과의 리그 경기에서 경기 후반전 도중 다리를 붙잡고 쓰러진 것으로 보였고 벤 화이트가 교체 출전했다.
아르테타 감독은 “그가 뭔가를 느꼈다. 피로 때문인지는 모르겠다. 우리는 곧바로 교체하기로 결정했고 위험을 감수하고 싶지 않았다. 그를 평가하고 상태를 지켜봐야 한다”라고 밝혔다.

'''

tok = Tokenizer()
tok.fit_on_texts([text]) #list타입이 되어야 한다. string을 리스트 형태로 넣어주어야 한다.
encoded= tok.texts_to_sequences([text])[0]
print(encoded)
print(tok.word_index) #{'눈이': 1, '거리에': 2, '하얀': 3, '쌓이고': 4, '있다': 5, '그의': 6, '밝게': 7, '빛난다': 8, '내리고': 9, '있는': 10, '오후': 11, '거리는': 12, '부시다': 13}

vocab_size= len(tok.word_index)+1 # vocab 사이즈로 1을 줬다.

In [ ]:
# 훈련 데이터
sequences = list()
for line in text.split('\n'): # 문장 토큰화
    enco= tok.texts_to_sequences([line])[0]
    # print(enco) #[2, 3, 1, 4, 5] [6, 1, 7, 8] [1, 9, 10, 11, 12, 1, 13]
    for i in range(1,len(enco)):
        sequ = enco[:i+1]
        # print(sequ)
# feature이랑 label 다음 단어를 제외시키며 학습시키는
        sequences.append(sequ)

print('학습에 참여할 샘플 수: %d'%len(sequences))
print(sequences)
max_len=max(len(i) for i in sequences) #7 -> 가장 긴 길이

psequences= pad_sequences(sequences,maxlen=max_len, padding='pre')
#가장 긴 길이에 맞추어야 계싼이 쉬움
print(psequences)

x= psequences[:,:-1]
y= psequences[:,-1]
print(x)
print(y)
#다항 분류일때는 label을 one hot encoding을 해주어야한다.
y= to_categorical(y, num_classes=vocab_size)
print(y[:2])

In [ ]:
model= Sequential()
model.add(Embedding(vocab_size, 32, input_length=max_len-1))
model.add(LSTM(32,activation='tanh'))
model.add(Flatten())
model.add(Dense(32,activation='tanh'))
model.add(Dense(32,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))
print(model.summary())
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x,y, epochs=200, verbose=2)
print('model.evaluate :', model.evaluate(x,y))

In [ ]:
# 문자열 생성
def sequence_gen_func(model, t, current_word,n):
    init_word=current_word
    sentence=''
    for _ in range(n):
        encoded=t.texts_to_sequences([current_word])[0]
        encoded=pad_sequences([encoded], maxlen=max_len-1, padding='pre')
        result=np.argmax(model.predict(encoded, verbose=0), axis=-1)
        #예측 단어 찾기
        for word, index in t.word_index.items():
            # print(word, index)
            if index == result: # 예측한 단어의 인덱스와 동일한 단어가 있다면
                break #해당 단어가 예측단어이므로 break
            current_word = current_word + ' '+ word
            sentence = sentence + ' '+ word
        sentence= init_word+sentence
        return sentence



'''
print(sequence_gen_func(model, tok,'눈이',5))
print(sequence_gen_func(model, tok,'그의',3))
print(sequence_gen_func(model, tok,'그의',10))
print(sequence_gen_func(model, tok,'거리에',3))
'''

print(sequence_gen_func(model, tok,'아스널',5))
print(sequence_gen_func(model, tok,'토미야스',3))
print(sequence_gen_func(model, tok,'토미야스',10))
print(sequence_gen_func(model, tok,'아르테타',3))
